In [2]:
from itertools import combinations,permutations
from collections import deque
import numpy as np

In [18]:
class Frequent_items:
    """Implementation of the Apriori algorithm for frequent itemsets detection proposed
    in the paper 'Fast Algorithms for Mining Association Rules' by R. Agrawal and R. Srikant.
    """
    def __init__(self, filepath):
        """
        Initialization method that receives a filepath from which read information.
        
        filepath:   Path to the file containing the transactions (one per line),
                    each represented as a set of integers separated by spaces.
        """
        with open(filepath, "r") as f:
            self.transactions = {}
            for i,line in enumerate(f.readlines()):
                for elem in line.split():
                    try:
                        elem = int(elem)
                    except ValueError:
                        pass
                    if elem not in self.transactions:
                        self.transactions[elem] = set()
                    self.transactions[elem].add(i)
            self.c1 = [({elem}, len(indices)) for elem, indices in self.transactions.items()]

    def _get_support(self, itemsets):
        """
        This method computes the support of a collection of itemsets based on the
        transactions.
        
        itemsets: An iterable of Python sets representing itemsets.
        returns:  An iterable of 2-tuples where the first element is
                  a set (representing an itemset) and the second one
                  is an integer (representing the support).
        """
        # Initialize counts
        supports = deque()
        for iset in itemsets:
            common_indices = set.intersection(*[self.transactions[item] for item in iset])
            supports.append((iset, len(common_indices)))
        return supports
    
    def _next_candidates(self, lprev):
        """
        Find the set of candidates based on the previous frequent
        (k-1)-itemsets.
        
        lprev:      The collection of previous large (k-1)-itemsets as
                    an an iterable of sets.
        returns:    An iterable of 2-tuples where the first element is
                    a set (representing an itemset) and the second one
                    is an integer (representing the support).
                
        """
        lprev = [set(itemset) for itemset in lprev.keys()]
        k = len(lprev[0])+1
        
        # Join (k-1)-itemsets to get all candidates of size k
        allcandidates = [s1 | s2 for s1 in lprev for s2 in lprev
                                 if len(s1 | s2) == k]
        
        # Filter out candidates which have some (k-1) subset not
        # identified as a large (k-1)-itemset
        candidates = deque()
        for iset in allcandidates:
            # Compute subsets of k-1 elements
            # Number of (k-1) combinations for a set of
            # k elements is precisely k (binomial coefficient(k,k-1))
            # so len(subsets) is k
            subsets = [set(x) for x in combinations(iset, k-1)]
            for i in range(k):
                if subsets[i] not in lprev:
                    break
                if i == k-1 and iset not in candidates: # Last iteration
                    candidates.append(iset)
            
        # Return candidates with their corresponding support
        return self._get_support(candidates)

    def _filter_candidates(self, candidates):
        """
        This methods select only those candidates such that their
        support is greater than or equal the support threshold.
        
        candidates: An iterable of 2-tuples where the first element is
                    a set (representing an itemset) and the second one
                    is an integer (representing the support).
        returns:    An iterable of itemsets as sets.
        """
        # Filter out itemsets with low support
        return {tuple(itemset):sup for itemset,sup in candidates
                        if sup >= self.minsup}
    
    def get_frequent_items(self, minsup):
        """
        Get the frequent items of the loaded transactions based on the
        provided support threshold.
        
        minsup:    Support threshold for the itemsets filtering.
        returns:   The identified frequent items as a set of tuples.
        """
        # Initialize variables
        self.minsup = minsup
        l = fi._filter_candidates(self.c1)
        answer = l
        
        # Updates candidates and answer
        while l:
            ck = fi._next_candidates(l)
            l = fi._filter_candidates(ck)
            answer.update(l)
        
        return answer
    
    def _get_confidence(self, union_supp, pre_supp):
        return union_supp/pre_supp
    
    def get_rules(self, min_confidence, itemsets):
        relevant_items = {}
        for item,support in itemsets.items():
            if len(item) > 1:
                
                # Add >2-tuple value
                relevant_items[frozenset(item)] = support
                
                # Add 1-tuple values
                relevant_items[frozenset([item[0]])] = itemsets[tuple([item[0]])]
                relevant_items[frozenset([item[1]])] = itemsets[tuple([item[1]])]
        
        possible_rules = permutations(relevant_items, 2)
        rules = []
        for rule in possible_rules:
            if set(rule[0]) & set(rule[1]) or frozenset(set(rule[0]) | set(rule[1])) not in relevant_items.keys():
                continue
            
            union_supp = relevant_items[frozenset(set(rule[0]) | set(rule[1]))]
            pre_supp = relevant_items[frozenset(rule[0])]

            if self._get_confidence(union_supp, pre_supp) >= min_confidence:
                rules.append((tuple(rule[0]),tuple(rule[1])))

        return rules

In [19]:
fi = Frequent_items("data/small_test.dat")
freq = fi.get_frequent_items(2)
print(freq)
fi.get_rules(1, freq)

{('E',): 3, ('C', 'A'): 2, ('A',): 2, ('C',): 3, ('E', 'C'): 2, ('B',): 3, ('E', 'C', 'B'): 2, ('B', 'E'): 3, ('B', 'C'): 2}


[(('E',), ('B',)),
 (('B', 'C'), ('E',)),
 (('E', 'C'), ('B',)),
 (('B',), ('E',)),
 (('A',), ('C',))]

In [5]:
fi = Frequent_items('data/T10I4D100K.dat')
fi.get_frequent_items(7000)

{(368,): 7828, (529,): 7057}

In [22]:
import cProfile, pstats, io
pr = cProfile.Profile()
pr.enable()
fi = Frequent_items('data/T10I4D100K.dat')
freq = fi.get_frequent_items(1000)
print(freq)
pr.disable()
s = io.StringIO()
sortby = 'time'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())
fi.get_rules(0, freq)

{(854,): 2847, (162,): 1450, (829,): 6810, (766,): 6265, (804,): 1315, (368,): 7828, (422,): 1255, (227, 390): 1049, (716,): 1199, (334,): 2146, (628,): 1102, (309,): 1262, (960,): 2732, (641,): 1494, (578,): 1290, (259,): 1522, (196,): 2096, (490,): 1066, (171,): 1097, (440,): 1943, (377,): 1149, (788,): 2386, (58,): 1330, (33,): 1460, (343,): 1599, (8,): 3090, (675,): 2976, (944,): 2794, (935,): 1742, (205,): 3605, (910,): 1695, (449,): 1890, (130,): 1711, (797,): 2684, (424,): 1448, (361,): 1104, (336,): 1071, (390,): 2685, (17,): 1683, (684,): 5408, (978,): 1141, (239,): 2742, (890,): 1437, (928,): 1034, (571,): 2902, (214,): 1893, (919,): 3710, (546,): 1050, (126,): 1075, (521,): 1582, (458,): 1124, (752,): 2578, (51,): 1612, (718,): 1238, (630,): 1523, (1,): 1535, (605,): 1652, (937,): 4681, (580,): 1667, (874,): 2237, (912,): 1009, (198,): 1461, (530,): 1263, (173,): 1080, (110,): 1801, (815,): 1358, (85,): 1555, (736,): 1470, (790,): 1094, (354,): 5835, (392,): 2420, (35,): 198

[((39,), (704,)),
 ((39,), (704, 825)),
 ((39,), (825,)),
 ((682,), (368,)),
 ((227,), (390,)),
 ((704, 39), (825,)),
 ((825, 39), (704,)),
 ((217,), (346,)),
 ((789,), (829,)),
 ((346,), (217,)),
 ((829,), (789,)),
 ((829,), (368,)),
 ((704,), (39,)),
 ((704,), (825, 39)),
 ((704,), (825,)),
 ((390,), (227,)),
 ((390,), (722,)),
 ((704, 825), (39,)),
 ((722,), (390,)),
 ((825,), (39,)),
 ((825,), (704, 39)),
 ((825,), (704,)),
 ((368,), (682,)),
 ((368,), (829,))]